# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
K. Doi  ->  K. Doi  |  ['K. Doi']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Arxiv has 73 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.19585


extracting tarball to tmp_2509.19585...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.19761
extracting tarball to tmp_2509.19761...

 done.


Issues with the citations
list index out of range
Retrieving document from  https://arxiv.org/e-print/2509.20040
extracting tarball to tmp_2509.20040...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.19761-b31b1b.svg)](https://arxiv.org/abs/2509.19761) | **Winding Motion of Spirals in a Gravitationally Unstable Protoplanetary Disk**  |
|| T. C. Yoshida, et al. -- incl., <mark>K. Doi</mark> |
|*Appeared on*| *2025-09-25*|
|*Comments*| *Published in Nature Astronomy on Sep 24, 2025, see this https URL. Supplementary Video is available at this https URL*|
|**Abstract**|            The discovery of wide-orbit giant exoplanets has posed a challenge to our conventional understanding of planet formation by coagulation of dust grains and planetesimals, and subsequent accretion of protoplanetary disk gas. As an alternative mechanism, the direct in-situ formation of planets or planetary cores by gravitational instability (GI) in protoplanetary disks has been proposed. However, observational evidence for GI in regions where wide-orbit planets are formed is still lacking. Theoretical studies predict that GI induces spiral arms moving at the local Keplerian speed in a disk. Here, with multiple high angular resolution observations over a seven-year time baseline using the Atacama Large Millimeter/submillimeter Array (ALMA), we report the evidence for spiral arms following the Keplerian rotation in the dust continuum disk around the young star IM Lup. This demonstrates that GI can operate in wide-orbit planet-formation regions, establishing it as a plausible formation mechanism for such planets.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.19585-b31b1b.svg)](https://arxiv.org/abs/2509.19585) | **NEXUS: A Search for Nuclear Variability with the First Two JWST NIRCam Epochs**  |
|| Z. Stone, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-09-25*|
|*Comments*| *16 pages, 10 figures, 1 table. Submitted to ApJ. The source catalog can be accessed here: this https URL*|
|**Abstract**|            The multi-cycle JWST Treasury program NEXUS will obtain cadenced imaging and spectroscopic observations around the North Ecliptic Pole during 2024-2028. Here we report a systematic search for nuclear variability among $\sim 25\,$k sources covered by NIRCam (F200W+F444W) imaging using the first two NEXUS epochs separated by 9 months in the observed frame. Difference imaging techniques reach $1\sigma$ variability sensitivity of 0.18~mag (F200W) and 0.15~mag (F444W) at 28th magnitude (within 0".2 diameter aperture), improved to $0.01$~mag and $0.02$~mag at $<25$th magnitude, demonstrating the superb performance of NIRCam photometry. The difference imaging results represent significant improvement over aperture photometry on individual epochs (by $>30\%$). We identify 465 high-confidence variable sources among the parent sample, with 2-epoch flux difference at $>3\sigma$ from the fiducial variability sensitivity. Essentially all these variable sources are of extragalactic origin based on preliminary photometric classifications, and follow a similar photometric redshift distribution as the parent sample up to $z_{\rm phot}>10$. While the majority of these variability candidates are likely normal unobscured AGNs, some of them may be rare nuclear stellar transients and tidal disruption events that await confirmation with spectroscopy and continued photometric monitoring. We also constrain the photometric variability of ten spectroscopically confirmed broad-line Little Red Dots (LRDs) at $3\lesssim z \lesssim 7$, and find none of them show detectable variability in either band. We derive stringent $3\sigma$ upper limits on the F444W variability of $\sim 3-10\%$ for these LRDs, with a median value of $\sim 5\%$. These constraints imply weak variability in the rest-frame optical continuum of LRDs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.20040-b31b1b.svg)](https://arxiv.org/abs/2509.20040) | **The role of photospheric magnetic flux diffusion in initiation of solar eruptions**  |
|| X. Bian, et al. -- incl., <mark>Y. Wang</mark>, <mark>Y. Wang</mark> |
|*Appeared on*| *2025-09-25*|
|*Comments*| *13 pages, 5 figures*|
|**Abstract**|            Solar eruptions may occur at different evolutionary stages of active regions, during which the photospheric motions manifest in various forms, including flux emergence, sunspot rotation, shearing, converging, and magnetic flux diffusion. However, it remains unclear what are the specific roles played by these different motions in leading to eruptions. Here, we employ high resolution magnetohydrodynamic simulations to demonstrate how solar eruptions can be initiated in a single bipolar configuration, driven by first shearing and then flux diffusion at the bottom surface. Flux diffusion disperses the photospheric magnetic flux, driving portions of it toward the polarity inversion line (PIL). This process leads to the expansion of core field, enhancing the pinching effect to form the current sheet. When magnetic reconnection occurs within this current sheet, the eruption is initiated, characterized by a rapid release of magnetic energy and accompanied by the formation of a erupting flux rope. Additionally, flux diffusion contributes to magnetic cancellation near the PIL, leading to the formation of a weakly twisted magnetic flux rope prior to the eruption. However, this pre-exist flux rope plays a limited role in eruption initiation, as its spatial position remains largely unchanged throughout the eruption. These findings demonstrate that the primary role of flux diffusion is to facilitate current sheet formation, highlighting the critical role of current sheet formation in eruption initiation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.19761/./deprojected.png', 'tmp_2509.19761/./deprojected_v3.png', 'tmp_2509.19761/./rot_curve.png']
copying  tmp_2509.19761/./deprojected.png to _build/html/
copying  tmp_2509.19761/./deprojected_v3.png to _build/html/
copying  tmp_2509.19761/./rot_curve.png to _build/html/
exported in  _build/html/2509.19761.md
    + _build/html/tmp_2509.19761/./deprojected.png
    + _build/html/tmp_2509.19761/./deprojected_v3.png
    + _build/html/tmp_2509.19761/./rot_curve.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\doi}[1]{\url{https://doi.org/#1}}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\doi}[1]{\url{https://doi.org/#1}}$
$\newcommand{\actaa}{Acta Astron.}$
$\newcommand{\araa}{Annu. Rev. Astron. Astrophys.}$
$\newcommand{\areps}{Annu. Rev. Earth Planet. Sci.}$
$\newcommand{\aar}{Astron. Astrophys. Rev.}$
$\newcommand{\ab}{Astrobiology}$
$\newcommand{\aj}{Astron. J.}$
$\newcommand{\ac}{Astron. Comput.}$
$\newcommand{\apart}{Astropart. Phys.}$
$\newcommand{\apj}{Astrophys. J.}$
$\newcommand{\apjl}{Astrophys. J. Lett.}$
$\newcommand{\apjs}{Astrophys. J. Suppl. Ser.}$
$\newcommand{\ao}{Appl. Opt.}$
$\newcommand{\apss}{Astrophys. Space Sci.}$
$\newcommand{\aap}{Astron. Astrophys.}$
$\newcommand{\aapr}{Astron. Astrophys. Rev.}$
$\newcommand{\aaps}{Astron. Astrophys. Suppl.}$
$\newcommand{\baas}{Bull. Am. Astron. Soc.}$
$\newcommand{\caa}{Chin. Astron. Astrophys.}$
$\newcommand{\cjaa}{Chin. J. Astron. Astrophys.}$
$\newcommand{\cqg}{Class. Quantum Gravity}$
$\newcommand{\epsl}{Earth Planet. Sci. Lett.}$
$\newcommand{\expa}{Exp. Astron.}$
$\newcommand{\frass}{Front. Astron. Space Sci.}$
$\newcommand{\gal}{Galaxies}$
$\newcommand{\gca}{Geochim. Cosmochim. Acta}$
$\newcommand{\grl}{Geophys. Res. Lett.}$
$\newcommand{\icarus}{Icarus}$
$\newcommand{\ija}{Int. J. Astrobiol.}$
$\newcommand{\jatis}{J. Astron. Telesc. Instrum. Syst.}$
$\newcommand{\jcap}{J. Cosmol. Astropart. Phys.}$
$\newcommand{\jgr}{J. Geophys. Res.}$
$\newcommand{\jgrp}{J. Geophys. Res.: Planets}$
$\newcommand{\jqsrt}{J. Quant. Spectrosc. Radiat. Transf.}$
$\newcommand{\lrca}{Living Rev. Comput. Astrophys.}$
$\newcommand{\lrr}{Living Rev. Relativ.}$
$\newcommand{\lrsp}{Living Rev. Sol. Phys.}$
$\newcommand{\memsai}{Mem. Soc. Astron. Italiana}$
$\newcommand{\maps}{Meteorit. Planet. Sci.}$
$\newcommand{\mnras}{Mon. Not. R. Astron. Soc.}$
$\newcommand{\nat}{Nature}$
$\newcommand{\nastro}{Nat. Astron.}$
$\newcommand{\ncomms}{Nat. Commun.}$
$\newcommand{\ngeo}{Nat. Geosci.}$
$\newcommand{\nphys}{Nat. Phys.}$
$\newcommand{\na}{New Astron.}$
$\newcommand{\nar}{New Astron. Rev.}$
$\newcommand{\physrep}{Phys. Rep.}$
$\newcommand{\pra}{Phys. Rev. A}$
$\newcommand{\prb}{Phys. Rev. B}$
$\newcommand{\prc}{Phys. Rev. C}$
$\newcommand{\prd}{Phys. Rev. D}$
$\newcommand{\pre}{Phys. Rev. E}$
$\newcommand{\prl}{Phys. Rev. Lett.}$
$\newcommand{\psj}{Planet. Sci. J.}$
$\newcommand{\planss}{Planet. Space Sci.}$
$\newcommand{\pnas}{Proc. Natl Acad. Sci. USA}$
$\newcommand{\procspie}{Proc. SPIE}$
$\newcommand{\pasa}{Publ. Astron. Soc. Aust.}$
$\newcommand{\pasj}{Publ. Astron. Soc. Jpn}$
$\newcommand{\pasp}{Publ. Astron. Soc. Pac.}$
$\newcommand{\raa}{Res. Astron. Astrophys.}$
$\newcommand{\rasti}{RAS Tech. Instrum.}$
$\newcommand{\rmxaa}{Rev. Mexicana Astron. Astrofis.}$
$\newcommand{\rnaas}{Res. Notes Am. Astron. Soc.}$
$\newcommand{\sci}{Science}$
$\newcommand{\sciadv}{Sci. Adv.}$
$\newcommand{\solphys}{Sol. Phys.}$
$\newcommand{\sovast}{Soviet Astron.}$
$\newcommand{\ssr}{Space Sci. Rev.}$
$\newcommand{\uni}{Universe}$
$\newcommand{\rv}[1]{{\color{orange}#1}}$
$\newcommand{\gretwo}{{\rm I\hspace{-1.2pt}I}}$
$\newcommand{\greone}{{\rm I}}$
$\newcommand{\farcs}{\mbox{\ensuremath{.\!\!^{\prime\prime}}}}$
$\newcommand{\figurename}{Supplementary Figure}$
$\newcommand{\tablename}{Supplementary Table}$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$
$\newcommand\url{#1}$
$\newcommand{\urlprefix}{URL }$</div>



<div id="title">

# Winding Motion of Spirals in a Gravitationally Unstable Protoplanetary Disk 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.19761-b31b1b.svg)](https://arxiv.org/abs/2509.19761)<mark>Appeared on: 2025-09-25</mark> -  _Published in Nature Astronomy on Sep 24, 2025, see this https URL. Supplementary Video is available at this https URL_

</div>
<div id="authors">

T. C. Yoshida, et al. -- incl., <mark>K. Doi</mark>

</div>
<div id="abstract">

**Abstract:** The discovery of wide-orbit giant exoplanets has posed a challenge to our conventional understanding of planet formation by coagulation of dust grains and planetesimals, and subsequent accretion of protoplanetary disk gas.As an alternative mechanism, the direct in-situ formation of planets or planetary cores by gravitational instability (GI) in protoplanetary disks has been proposed.However, observational evidence for GI in regions where wide-orbit planets are formed is still lacking.Theoretical studies predict that GI induces spiral arms moving at the local Keplerian speed in a disk.Here, with multiple high angular resolution observations over a seven-year time baseline using the Atacama Large Millimeter/submillimeter Array (ALMA), we report the evidence for spiral arms following the Keplerian rotation in the dust continuum disk around the young star IM Lup.This demonstrates that GI can operate in wide-orbit planet-formation regions, establishing it as a plausible formation mechanism for such planets.

</div>

<div id="div_fig1">

<img src="tmp_2509.19761/./deprojected.png" alt="Fig2" width="100%"/>

**Figure 2. -** Deprojected continuum images used in the quantitative analysis.
These images are normalized based on the 2017 epoch and shown in the unit of the corresponding brightness temperature (see Methods).
The blue solid curves indicate the spiral ridge \citep{huan18} at each epoch, assuming that they follow the local Keplerian rotation (i.e. the case of GI).
The blue dashed curves correspond to the spiral ridge in 2017.
The circles on the bottom left indicate the synthetic beam after deprojection.
 (*deproj*)

</div>
<div id="div_fig2">

<img src="tmp_2509.19761/./deprojected_v3.png" alt="Fig3" width="100%"/>

**Figure 3. -** Deprojected continuum images after partially subtracting the axisymmetric component.
The color scale is also stretched for visibility purposes (see Extended data for details).
The original data is the same as Figure \ref{deproj}.
The white contours are drawn at values of 2 and 4 (in an arbitrary unit).
The blue-filled scattered points follow the local Keplerian rotation, while the blue circles show the positions in 2017 for comparison. The rotation of the contour is consistent with the local Keplerian rotation.
 (*deproj2*)

</div>
<div id="div_fig3">

<img src="tmp_2509.19761/./rot_curve.png" alt="Fig4" width="100%"/>

**Figure 4. -** Rotation curve of the dust disk. $\omega(r)$ extracted with the image-fitting analysis is plotted in the left panel (blue line) while the radius-by-radius analysis result is shown in the right panel (blue circles). In the left panel, the blue solid line and the blue-shaded regions indicate the median and 16-84th percentiles of the marginal probability distribution at each radius, respectively.
The error bars in the right panel correspond to the $1\sigma$ range of the normalized probability distribution at each radius. Note that each point is not perfectly independent as the beam size is ${\sim}24$ au.
The dashed and dotted lines, respectively, show predictions for spirals induced by GI and a planet located at 117 au from the central star \citep{pint20}.
 (*rot_curve*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.19761"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

125  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
